# Community Deception

Connect Google Drive to access the dataset.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Install Pytorch Geometric

If we are on Kaggle we need to run the following cells to install Pytorch Geometric

In [ ]:
import torch
import os

os.environ["TORCH"] = torch.__version__

# On Colab we can have TORCH+CUDA on os.environ["TORCH"]

# Check if there is the cuda version on TORCH
if torch.cuda.is_available():
    print("CUDA is available")
    print(torch.version.cuda)
    if "+" not in os.environ["TORCH"]:
        os.environ["TORCH"] += "+cu" + \
            torch.version.cuda.replace(".", "")

print(os.environ["TORCH"])

Install torch geometric and optional dependencies:

In [ ]:
! pip install torch_geometric
# Optional dependencies:
# ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
# ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
# ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}.html
! pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-${TORCH}.html

# Graph
# ! pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
! pip install igraph
! pip install cdlib[C]
! pip install karateclub
# ! pip install graph2vec

**IMPORTANT!!!**
After the libraries installation, restart the runtime and start executing the cells below

## Import Libraries

In [ ]:
# Import torch and os another time to reset the colab enviroment after PyG installation
from IPython.display import FileLink, display
import subprocess
import torch
import os
import gc

# Typing
from typing import List, Tuple, Set, Callable
from collections import Counter, namedtuple

# Deep Learning
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch.distributions import MultivariateNormal

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import scipy

# Graph
from karateclub import GL2Vec, Graph2Vec, Node2Vec
from cdlib import algorithms
import cdlib
import networkx as nx
import igraph as ig

# cuGraph
# import cugraph as cnx


# Misc
from statistics import mean
from enum import Enum
from tqdm import trange
import math
import random
import json
import time

# Plot
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('default')

## Utils

In [ ]:
# Only for the notebook
TRAIN = True
# Set to True to test the results with the baselines algorithms
TEST = False

In [ ]:
class FilePaths(Enum):
    """Class to store file paths for data and models"""
    # ° Local
    # DATASETS_DIR = 'dataset/data'
    # LOG_DIR    = 'src/logs/'
    # TEST_DIR = 'test/'
    # ° Kaggle
    DATASETS_DIR = '/kaggle/input/network-community'
    LOG_DIR = '/kaggle/working/logs/'
    TEST_DIR = '/kaggle/working/test/'
    # ° Google Colab
    # DATASETS_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Datasets"
    # LOG_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Logs/"
    # TEST_DIR = "/content/drive/MyDrive/Sapienza/Tesi/Test/"
    
    # ! Trained model path for testing (change the following line to change the model)
    TRAINED_MODEL = "/kaggle/input/test-community-deception-model/node2vec_gcnconv.pth"
    # Dataset file paths
    KAR = DATASETS_DIR + '/kar.mtx'
    DOL = DATASETS_DIR + '/dol.mtx'
    MAD = DATASETS_DIR + '/mad.mtx'
    LESM = DATASETS_DIR + '/lesm.mtx'
    POLB = DATASETS_DIR + '/polb.mtx'
    WORDS = DATASETS_DIR + '/words.mtx'
    ERDOS = DATASETS_DIR + '/erdos.mtx'
    POW = DATASETS_DIR + '/pow.mtx'
    FB_75 = DATASETS_DIR + '/fb-75.mtx'
    DBLP = DATASETS_DIR + '/dblp.mtx'
    ASTR = DATASETS_DIR + '/astr.mtx'
    AMZ = DATASETS_DIR + '/amz.mtx'
    YOU = DATASETS_DIR + '/you.mtx'
    ORK = DATASETS_DIR + '/ork.mtx'


class DetectionAlgorithmsNames(Enum):
    """
    Enum class for the detection algorithms
    """
    LOUV = "louvain"
    WALK = "walktrap"
    GRE = "greedy"
    INF = "infomap"
    LAB = "label_propagation"
    EIG = "eigenvector"
    BTW = "edge_betweenness"
    SPIN = "spinglass"
    OPT = "optimal"
    SCD = "scalable_community_detection"


class SimilarityFunctionsNames(Enum):
    """
    Enum class for the similarity functions
    """
    # Community similarity functions
    JAC = "jaccard"
    OVE = "overlap"
    SOR = "sorensen"
    # Graph similarity functions
    GED = "ged"  # Graph edit distance
    JAC_1 = "jaccard_1"
    JAC_2 = "jaccard_2"


class HyperParams(Enum):
    """Hyperparameters for the Environment"""
    # ! REAL GRAPH Graph path (change the following line to change the graph)
    GRAPH_NAME = None # FilePaths.LESM.value
    # ! Define the detection algorithm to use (change the following line to change the algorithm)
    DETECTION_ALG_NAME = DetectionAlgorithmsNames.GRE.value
    # Multiplier for the rewiring action number, i.e. (mean_degree * BETA)
    BETA = 30
    # ! Strength of the deception constraint, value between 0 (hard) and 1 (soft) 
    TAU = 0.99
    # ° Hyperparameters  Testing ° #
    # ! Weight to balance the penalty in the reward
    # The higher its value the more importance the penalty will have
    LAMBDA = [0.1, 1] # [0.01, 0.1, 1]
    # ! Weight to balance the two metrics in the definition of the penalty
    # The higher its value the more importance the distance between communities 
    # will have, compared with the distance between graphs
    ALPHA = [0.7] # [0.3, 0.5, 0.7]
    # Multiplier for the number of maximum steps allowed
    MAX_STEPS_MUL = 2
    
    """ Graph Encoder Parameters """""
    EMBEDDING_DIM = 128 # 256

    """ Agent Parameters"""
    # Networl Architecture
    HIDDEN_SIZE_1 = 64
    HIDDEN_SIZE_2 = 64
    # Hyperparameters for the ActorCritic
    EPS_CLIP = np.finfo(np.float32).eps.item()  # 0.2
    BEST_REWARD = -np.inf
    # ° Hyperparameters  Testing ° #
    # ! Learning rate, it controls how fast the network learns
    LR = [1e-4] # [1e-7, 1e-4, 1e-1]
    # ! Discount factor
    GAMMA = [0.3, 0.7] # [0.9, 0.95]
    
    """ Training Parameters """
    # Number of episodes to collect experience
    MAX_EPISODES = 1000
    # Dictonary for logging
    LOG_DICT = {
        # List of rewards per episode
        'train_reward_list': [],
        # Avg reward per episode, with the last value multiplied per 10 if the 
        # goal is reached
        'train_reward_mul': [],
        # Total reward per episode
        'train_reward': [],
        # Number of steps per episode
        'train_steps': [],
        # Average reward per episode
        'train_avg_reward': [],
        # Average Actor loss per episode
        'a_loss': [],
        # Average Critic loss per episode
        'v_loss': [],
        # set max number of training episodes
        'train_episodes': MAX_EPISODES,
    }
    
    """Evaluation Parameters"""
    # ! Change the following parameters according to the hyperparameters to test
    STEPS_EVAL = 100
    LR_EVAL = LR[0]
    GAMMA_EVAL = GAMMA[0]
    LAMBDA_EVAL = LAMBDA[0]
    ALPHA_EVAL = ALPHA[0]
    # Algorithms to evaluate
    ALGS_EVAL = ["Roam",  "Random", "Degree", "Agent"]
    # Metrics for each algorithm
    METRICS_EVAL = ["goal", "nmi", "time", "steps"]
    
    """Graph Generation Parameters"""
    # ! Change the following parameters to modify the graph
    # Number of nodes
    N_NODE = 200
    # Power law exponent for the degree distribution of the created graph.
    TAU1 = 2
    # Power law exponent for the community size distribution in the created graph.
    TAU2 = 1.1
    # Fraction of inter-community edges incident to each node.
    MU = 0.1

    # Desired average degree of nodes in the created graph.
    AVERAGE_DEGREE = int(0.05 * N_NODE)  # 20
    # Minimum degree of nodes in the created graph
    MIN_DEGREE = None  # 30
    # Maximum degree of nodes in the created graph
    MAX_DEGREE = int(0.19 * N_NODE)

    # Minimum size of communities in the graph.
    MIN_COMMUNITY = int(0.05 * N_NODE)
    # Maximum size of communities in the graph.
    MAX_COMMUNITY = int(0.2 * N_NODE)

    # Maximum number of iterations to try to create the community sizes, degree distribution, and community affiliations.
    MAX_ITERS = 5000
    # Seed for the random number generator.
    SEED = 10


class Utils:
    """Class to store utility functions"""

    @staticmethod
    def import_mtx_graph(file_path: str) -> nx.Graph:
        """
        Import a graph from a .mtx file

        Parameters
        ----------
        file_path : str
            File path of the .mtx file

        Returns
        -------
        nx.Graph
            Graph imported from the .mtx file
        """
        try:
            graph_matrix = scipy.io.mmread(file_path)
            graph = nx.Graph(graph_matrix)
            for node in graph.nodes:
                # graph.nodes[node]['name'] = node
                graph.nodes[node]['num_neighbors'] = len(
                    list(graph.neighbors(node)))
            return graph
        except Exception as exception:
            print("Error: ", exception)
            return None
    
    @staticmethod
    def generate_lfr_benchmark_graph(
        n: int=HyperParams.N_NODE.value,
        tau1: float=HyperParams.TAU1.value,
        tau2: float=HyperParams.TAU2.value,
        mu: float=HyperParams.MU.value,   
        average_degree: int = HyperParams.AVERAGE_DEGREE.value,
        min_degree: int=HyperParams.MIN_DEGREE.value,
        max_degree: int=HyperParams.MAX_DEGREE.value,
        min_community: int=HyperParams.MIN_COMMUNITY.value,
        max_community: int=HyperParams.MAX_COMMUNITY.value,
        max_iters: int=HyperParams.MAX_ITERS.value,
        seed: int=HyperParams.SEED.value)->Tuple[nx.Graph, str]:
        """
        Generate a LFR benchmark graph for community detection algorithms.

        Parameters
        ----------
        n : int, optional
            Number of nodes, by default 500
        tau1 : float, optional
            _description_, by default 3
        tau2 : float, optional
            _description_
        mu : float, optional
            Mixing parameter, by default 0.1
        average_degree : int, optional
            Average degree of the nodes, by default 20
        min_degree : int, optional
            Minimum degree of the nodes, by default 20
        max_degree : int, optional
            Maximum degree of the nodes, by default 50
        min_community : int, optional
            Minimum number of communities, by default 10
        max_community : int, optional
            Maximum number of communities, by default 50
        max_iters : int, optional
            Maximum number of iterations, by default 5000
        seed : int, optional
            Seed for the random number generator, by default 10

        Returns
        -------
        nx.Graph
            Synthetic graph generated with the LFR benchmark
        file_path : str
            Path to the file where the graph is saved
        """
        graph = nx.generators.community.LFR_benchmark_graph(
            n=n,
            tau1=tau1,
            tau2=tau2,
            mu=mu,
            average_degree=average_degree,
            min_degree=min_degree,
            max_degree=max_degree,
            min_community=min_community,
            max_community=max_community,
            max_iters=max_iters,
            seed=seed)
        # file_path = FilePaths.DATASETS_DIR.value + f"/lfr_benchmark_node-{n}.mtx"
        # ! FOR KAGGLE NOTEBOOK
        file_path = f"/kaggle/working/lfr_benchmark_node-{n}.mtx"
        nx.write_edgelist(graph, file_path, data=False)
        # Delete community attribute from the nodes to handle PyG compatibility
        for node in graph.nodes:
            if 'community' in graph.nodes[node]:
                del graph.nodes[node]['community']
        for edge in graph.edges:
            graph.edges[edge]['weight'] = 1
        return graph, file_path
        
    @staticmethod
    def check_dir(path: str):
        """
        Check if the directory exists, if not create it.

        Parameters
        ----------
        path : str
            Path to the directory
        """
        if not os.path.exists(path):
            os.makedirs(path)
    
    @staticmethod
    def plot_training(
        log: dict, 
        env_name: str, 
        detection_algorithm: str,
        file_path: str,
        window_size: int=int(HyperParams.MAX_EPISODES.value/100)):
        """Plot the training results

        Parameters
        ----------
        log : dict
            Dictionary containing the training logs
        env_name : str
            Name of the environment
        detection_algorithm : str
            Name of the detection algorithm
        file_path : str
            Path to save the plot
        window_size : int, optional
            Size of the rolling window, by default 100
        """
        def plot_seaborn(
                df: pd.DataFrame,
                path: str,
                env_name: str,
                detection_algorithm: str,
                labels: Tuple[str, str],
                colors: Tuple[str, str]) -> None:
            sns.set_style("darkgrid")
            sns.lineplot(data=df, x="Episode", y=labels[0], color=colors[0])
            sns.lineplot(data=df, x="Episode", y=labels[1], color=colors[1],
                        estimator="mean", errorbar=None)
            plt.title(
                f"Training on {env_name} graph with {detection_algorithm} algorithm")
            plt.xlabel("Episode")
            plt.ylabel(labels[0])
            plt.savefig(path)
            plt.clf()
        
        if window_size < 1:
            window_size = 1
        df = pd.DataFrame({
            "Episode": range(len(log["train_avg_reward"])),
            "Avg Reward": log["train_avg_reward"],
            "Steps per Epoch": log["train_steps"],
            "Goal Reward": log["train_reward_mul"],
            "Goal Reached": [1/log["train_steps"][i] if log["train_reward_list"][i][-1]
                > 1 else 0 for i in range(len(log["train_steps"]))],
        })
        df["Rolling_Avg_Reward"] = df["Avg Reward"].rolling(window_size).mean()
        df["Rolling_Steps"] = df["Steps per Epoch"].rolling(window_size).mean()
        df["Rolling_Goal_Reward"] = df["Goal Reward"].rolling(window_size).mean()
        df["Rolling_Goal_Reached"] = df["Goal Reached"].rolling(window_size).mean()
        plot_seaborn(
            df,
            file_path+"/training_reward.png",
            env_name,
            detection_algorithm,
            ("Avg Reward", "Rolling_Avg_Reward"),
            ("lightsteelblue", "darkblue"),
        )
        plot_seaborn(
            df,
            file_path+"/training_steps.png",
            env_name,
            detection_algorithm,
            ("Steps per Epoch", "Rolling_Steps"),
            ("thistle", "purple"),
        )
        plot_seaborn(
            df,
            file_path+"/training_goal_reward.png",
            env_name,
            detection_algorithm,
            ("Goal Reward", "Rolling_Goal_Reward"),
            ("darkgray", "black"),
        )
        plot_seaborn(
            df,
            file_path+"/training_goal_reached.png",
            env_name,
            detection_algorithm,
            ("Goal Reached", "Rolling_Goal_Reached"),
            ("darkgray", "black"),
        )

        df = pd.DataFrame({
            "Episode": range(len(log["a_loss"])),
            "Actor Loss": log["a_loss"],
            "Critic Loss": log["v_loss"],
        })
        df["Rolling_Actor_Loss"] = df["Actor Loss"].rolling(window_size).mean()
        df["Rolling_Critic_Loss"] = df["Critic Loss"].rolling(window_size).mean()
        plot_seaborn(
            df,
            file_path+"/training_a_loss.png",
            env_name,
            detection_algorithm,
            ("Actor Loss", "Rolling_Actor_Loss"),
            ("palegreen", "darkgreen"),
        )
        plot_seaborn(
            df,
            file_path+"/training_v_loss.png",
            env_name,
            detection_algorithm,
            ("Critic Loss", "Rolling_Critic_Loss"),
            ("lightcoral", "darkred"),
        )

        
    ############################################################################
    #                               EVALUATION                                 #
    ############################################################################   
    @staticmethod   
    def get_new_community(
        node_target: int,
        new_community_structure: List[List[int]]) -> List[int]:
        """
        Search the community target in the new community structure after 
        deception. As new community target after the action, we consider the 
        community that contains the target node, if this community satisfies 
        the deception constraint, the episode is finished, otherwise not.

        Parameters
        ----------
        node_target : int
            Target node to be hidden from the community
        new_community_structure : List[List[int]]
            New community structure after deception

        Returns
        -------
        List[int]
            New community target after deception
        """
        for community in new_community_structure.communities:
            if node_target in community:
                return community
        raise ValueError("Community not found")
    
    @staticmethod
    def check_goal(
            env,#: GraphEnvironment,
            node_target: int,
            old_community: int,
            new_community: int) -> int:
        """
        Check if the goal of hiding the target node was achieved

        Parameters
        ----------
        env : GraphEnvironment
            Environment of the agent
        node_target : int
            Target node
        old_community : int
            Original community of the target node
        new_community : int
            New community of the target node
        similarity_function : Callable
            Similarity function to use
            
        Returns
        -------
        int
            1 if the goal was achieved, 0 otherwise
        """
        if len(new_community) == 1:
            return 1
        # Copy the communities to avoid modifying the original ones
        new_community_copy = new_community.copy()
        new_community_copy.remove(node_target)
        old_community_copy = old_community.copy()
        old_community_copy.remove(node_target)
        # Compute the similarity between the new and the old community
        similarity = env.community_similarity(
            new_community_copy,
            old_community_copy
        )
        del new_community_copy, old_community_copy
        if similarity <= env.tau:
            return 1
        return 0

    @staticmethod
    def initialize_dict(algs: List[str])->dict:
        """
        Initialize the dictionary for the evaluation

        Parameters
        ----------
        algs : List[str]
            List of algorithms names to evaluate
        
        Returns
        -------
        dict
            Dictionary for the evaluation, where the keys are the algorithms
            names and the values are dictionaries containing the metrics
        """
        log_dict = dict()
        
        for alg in algs:
            log_dict[alg] = {
                "goal": [],
                "nmi": [],
                "time": [],
                "steps": [],
            }
        return log_dict
    
    @staticmethod
    def save_test(
        log: dict, 
        files_path: str, 
        log_name: str, 
        algs: List[str],
        metrics: List[str]):
        """Save and Plot the testing results

        Parameters
        ----------
        log : dict
            Dictionary containing the training logs
        files_path : str
            Path to save the plot
        log_name : str
            Name of the log file
        algs : List[str]
            List of algorithms names to evaluate
        metrics : List[str]
            List of metrics to evaluate
        """
        file_name = f"{files_path}/{log_name}.json"
        # Save json file
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(log, f, indent=4)
            
        for metric in metrics:
            # Create a DataFrame with the mean values of each algorithm for the metric
            df = pd.DataFrame({
                "Algorithm": algs,
                metric.capitalize(): [mean(log[alg][metric]) for alg in algs]
            })
            
            # Convert the goal column to percentage
            if metric == "goal":
                df[metric.capitalize()] = df[metric.capitalize()] * 100

            sns.barplot(data=df,
                        x="Algorithm",
                        y=metric.capitalize(),
                        palette=sns.color_palette("Set1"))
            plt.title(f"Evaluation on {log['env']['dataset']} graph with {log['env']['detection_alg']} algorithm")
            plt.xlabel("Algorithm")
            if metric == "goal":
                plt.ylabel(f"{metric.capitalize()} reached %")
            elif metric == "time":
                plt.ylabel(f"{metric.capitalize()} (s)")
            else:
                plt.ylabel(metric.capitalize())
            plt.savefig(f"{files_path}/{log_name}_{metric}.png")
            plt.clf()

In [ ]:
# Create paths
Utils.check_dir(FilePaths.LOG_DIR.value)
Utils.check_dir(FilePaths.TEST_DIR.value)

## Community Algorithms

### Community Detection

In [ ]:
class CommunityDetectionAlgorithm(object):
    """Class for the community detection algorithms using CDLIB"""
    def __init__(self, alg_name: str) -> None:
        """
        Initialize the DetectionAlgorithm object

        Parameters
        ----------
        alg_name : str
            The name of the algorithm
        """
        self.alg_name = alg_name

    def compute_community(self, graph: nx.Graph) -> cdlib.NodeClustering:
        """Compute the community partition of the graph

        Parameters
        ----------
        graph : nx.Graph
            Input graph

        Returns
        -------
        cdlib.NodeClustering
            Cdlib NodeClustering object
        """
        # Rename DetectionAlgorithms Enum to da for convenience
        da = DetectionAlgorithmsNames
        # Choose the algorithm
        if self.alg_name == da.LOUV.value:
            return algorithms.louvain(graph)
        elif self.alg_name == da.WALK.value:
            return algorithms.walktrap(graph)
        elif self.alg_name == da.GRE.value:
            return algorithms.greedy_modularity(graph)
        elif self.alg_name == da.INF.value:
            return algorithms.infomap(graph)
        # elif self.alg_name == da.LAB.value:
        #    # ! Return a EdgeClustering object
        #    return algorithms.label_propagation(graph)
        elif self.alg_name == da.EIG.value:
            return algorithms.eigenvector(graph)
        # elif self.alg_name == da.BTW.value:
        #     return self.compute_btw(graph, args)
        elif self.alg_name == da.SPIN.value:
            return algorithms.spinglass(graph)
        # elif self.alg_name == da.OPT.value:
        #    return self.compute_opt(graph, args)
        # elif self.alg_name == da.SCD.value:
        #    return self.compute_scd(graph)
        else:
            raise ValueError('Invalid algorithm name')

### Community Deception Baselines

#### Random Hiding

In [ ]:

class RandomHiding():
    
    def __init__(
        self, 
        env, 
        steps: int, 
        target_community: List[int]):
        self.env = env
        self.graph = self.env.original_graph
        self.steps = steps
        self.target_node = self.env.node_target
        self.target_community = target_community
        self.detection_alg = self.env.detection
        self.original_community_structure = self.env.original_community_structure
        self.possible_edges = self.get_possible_action() # self.env.possible_actions
        # Put all the edges in a list
        # self.possible_edges = self.env.possible_actions
        # self.possible_edges = list(self.possible_edges["ADD"]) + list(self.possible_edges["REMOVE"])
        
    def get_possible_action(self):
        # Put all edge between the target node and its neighbors in a list
        possible_actions_add = []
        for neighbor in self.graph.neighbors(self.target_node):
            possible_actions_add.append((self.target_node, neighbor))
        
        # Put all the edges that aren't neighbors of the target node in a list
        possible_actions_remove = []
        for node in self.graph.nodes():
            if node != self.target_node and node not in self.graph.neighbors(self.target_node):
                possible_actions_remove.append((self.target_node, node))
        possible_action = possible_actions_add + possible_actions_remove
        return possible_action
    
    def hide_target_node_from_community(self)->tuple:
        """
        Hide the target node from the target community by rewiring its edges, 
        choosing randomly between adding or removing an edge.
        
        Returns
        -------
        G_prime: nx.Graph
        """
        graph = self.graph.copy()
        done = False
        while self.steps > 0 and not done:
            # Random choose a edge from the possible edges
            edge = self.possible_edges.pop()
            if graph.has_edge(*edge):
                # Remove the edge
                graph.remove_edge(*edge)
            else:
                # Add the edge
                graph.add_edge(*edge)
            
            # Compute the new community structure
            communities = self.detection_alg.compute_community(graph)
            new_community = Utils.get_new_community(
                self.target_node, communities)

            check = Utils.check_goal(
                self.env, self.target_node, self.target_community, new_community)
            if check == 1:
                # If the target community is a subset of the new community, the episode is finished
                done = True
            self.steps -= 1
            
            self.steps -= 1
        return graph, communities

#### Degree Hiding

In [ ]:
class DegreeHiding():

    def __init__(
            self,
            env,
            steps: int,
            target_community: List[int]):
        self.env = env
        self.graph = self.env.original_graph
        self.steps = steps
        self.target_node = self.env.node_target
        self.target_community = target_community
        self.detection_alg = self.env.detection
        self.original_community_structure = self.env.original_community_structure
        self.possible_edges = self.get_possible_action()  # self.env.possible_actions
        # Put all the edges in a list
        # self.possible_edges = self.env.possible_actions
        # self.possible_edges = list(self.possible_edges["ADD"]) + list(self.possible_edges["REMOVE"])

    def get_possible_action(self):
        # Put all edge between the target node and its neighbors in a list
        possible_actions_add = []
        for neighbor in self.graph.neighbors(self.target_node):
            possible_actions_add.append((self.target_node, neighbor))

        # Put all the edges that aren't neighbors of the target node in a list
        possible_actions_remove = []
        for node in self.graph.nodes():
            if node != self.target_node and node not in self.graph.neighbors(self.target_node):
                possible_actions_remove.append((self.target_node, node))
        possible_action = possible_actions_add + possible_actions_remove
        return possible_action
    
    def hide_target_node_from_community(self) -> tuple:
        """
        Hide the target node from the target community by rewiring its edges, 
        choosing the node with the highest degree between adding or removing an edge.
        
        Returns
        -------
        G_prime: nx.Graph
        """
        graph = self.graph.copy()
        done = False
        # From the list possible_edges, create a list of tuples 
        # (node1, node2, degree_of_node2)
        possible_edges = []
        for edge in self.possible_edges:
                possible_edges.append(
                    (edge[0], edge[1], graph.degree(edge[1])))
        while self.steps > 0 and not done:
            # Choose the edge with the highest degree
            max_tuple = max(possible_edges, key=lambda x: x[2])
            possible_edges.remove(max_tuple)
            edge = (max_tuple[0], max_tuple[1])
            
            if graph.has_edge(*edge):
                # Remove the edge
                graph.remove_edge(*edge)
            else:
                # Add the edge
                graph.add_edge(*edge)

            # Compute the new community structure
            communities = self.detection_alg.compute_community(graph)
            new_community = Utils.get_new_community(self.target_node, communities)

            check = Utils.check_goal(self.env, self.target_node, self.target_community, new_community)
            if check == 1:
                # If the target community is a subset of the new community, the episode is finished
                done = True
            self.steps -= 1
        return graph, communities

#### Roam Hiding

In [ ]:
class RoamHiding():
    """Given a network and a source node v,our objective is to conceal the 
    importance of v by decreasing its centrality without compromising its
    influence over the network.
    
    From the article "Hiding Individuals and Communities in a Social Network".
    """
    def __init__(self, graph: nx.Graph, target_node: int, detection_alg: str) -> None:
        self.graph = graph
        self.target_node = target_node
        self.detection_alg = CommunityDetectionAlgorithm(detection_alg)
    
    @staticmethod
    def get_edge_budget(graph: nx.Graph, budget: float) -> int:
        """
        Compute the number of edges to add given a budget and a graph.

        Parameters
        ----------
        graph : nx.Graph
            Graph to add edges to.
        budget : int
            Budget of the attack, value between 0 and 100.

        Returns
        -------
        int
            Number of edges to add.
        """
        assert budget > 0 and budget <= 100, "Budget must be between 0 and 100"
        return int(budget * graph.number_of_edges() / 100)
    
    def roam_heuristic(self, budget: int) -> tuple:
        """
        The ROAM heuristic given a budget b:
            - Step 1: Remove the link between the source node, v, and its 
            neighbour of choice, v0;
            - Step 2: Connect v0 to b − 1 nodes of choice, who are neighbours 
            of v but not of v0 (if there are fewer than b − 1 such neighbours, 
            connect v0 to all of them).

        Returns
        -------
        graph : nx.Graph
            The graph after the ROAM heuristic.
        """
        edge_budget = self.get_edge_budget(self.graph, budget)
        
        # ° --- Step 1 --- ° #
        target_node_neighbours = list(self.graph.neighbors(self.target_node))
        
        # Choose v0 as the neighbour of target_node with the most connections
        v0 = target_node_neighbours[0]
        for v in target_node_neighbours:
            if self.graph.degree[v] > self.graph.degree[v0]:
                v0 = v
        # v0 = random.choice(target_node_neighbours)    # Random choice
        # Remove the edge between v and v0
        self.graph.remove_edge(self.target_node, v0)
        
        # ° --- Step 2 --- ° #
        # Get the neighbours of v0
        v0_neighbours = list(self.graph.neighbors(v0))
        # Get the neighbours of v, who are not neighbours of v0
        v_neighbours_not_v0 = [x for x in target_node_neighbours if x not in v0_neighbours]
        # If there are fewer than b-1 such neighbours, connect v_0 to all of them
        if len(v_neighbours_not_v0) < edge_budget-1:
            edge_budget = len(v_neighbours_not_v0) + 1
        # Make an ascending order list of the neighbours of v0, based on their degree
        sorted_neighbors = sorted(v_neighbours_not_v0, key=lambda x: self.graph.degree[x]) 
        # Connect v_0 to b-1 nodes of choice, who are neighbours of v but not of v_0
        for i in range(edge_budget-1):
            v0_neighbour = sorted_neighbors[i]
            # v0_neighbour = random.choice(v_neighbours_not_v0)   # Random choice
            self.graph.add_edge(v0, v0_neighbour)
            v_neighbours_not_v0.remove(v0_neighbour)
        
        new_community_structure = self.detection_alg.compute_community(self.graph)
        return self.graph, new_community_structure

### Community Deception Metrics & Algorithms

#### Deception Score

In [ ]:
class DeceptionScore(object):
    """Deception score of a community detection algorithm."""
    def __init__(self, community_target: List[int]) -> None:
        self.community_target = community_target

    @staticmethod
    def recall(g_i: List[int], community_target: List[int]) -> float:
        """Calculate recall score of a community g_i

        Parameters
        ----------
        g_i : List[int]
            Community found by a community detection algorithm.

        Returns
        -------
        float
            Recall score of g_i.
        """
        # Number of members in g_i that are also in our community
        members_in_g_i = len(set(community_target) & set(g_i))
        return members_in_g_i / len(community_target)

    @staticmethod
    def precision(g_i: List[int], community_target: List[int]) -> float:
        """Calculate precision score of a community g_i

        Parameters
        ----------
        g_i : List[int]
            Community found by a community detection algorithm.

        Returns
        -------
        float
            Precision score of g_i.
        """
        # Number of members in G_i that are also in our community
        members_in_g_i = len(set(community_target) & set(g_i))
        return members_in_g_i / len(g_i)

    def compute_deception_score(
            self,
            community_structure: List[List[int]],
            connected_components: int) -> float:
        """Calculate deception score of a community detection algorithm.

        Parameters
        ----------
        community_structure : List(List(int))
            Community structure found by a community detection algorithm.
        connected_components : int
            Number of connected components in the graph.
        
        Returns
        -------
        deception_score : float
            Deception score of a community detection algorithm.
        """
        # Number of intersecting nodes between the community structure and community target
        n_intersecting_nodes = [g_i for g_i in community_structure if len(
            set(self.community_target) & set(g_i)) > 0]
        
        recall = max([self.recall(g_i, self.community_target) for g_i in community_structure])
        precision = sum([self.precision(g_i, self.community_target) for g_i in n_intersecting_nodes])
        
        # Ideal situation occurs when each member of the community target is 
        # placed in a different community and the value of the maximum recall 
        # is lower possible.
        assert len(self.community_target) - 1 > 0, "Community target must have at least 2 members."
        community_spread = 1 - (connected_components - 1) / (len(self.community_target) - 1)
        
        # Ideal situation occurs when each member of the community structure 
        # contains little percentage of the community target.
        assert len(n_intersecting_nodes) > 0, "Community structure must have at least 1 member."
        community_hiding = 0.5 * (1 - recall) + 0.5 * (1 - precision / len(n_intersecting_nodes))
        
        # Deception score is the product of community spread and community hiding. 
        deception_score = community_spread * community_hiding
        return deception_score

#### Safeness

In [ ]:
class Safeness:
    """Computes the safeness of a node in a community and the safeness of a community."""
    def __init__(self, graph: nx.Graph, community_target: List[int]):
        self.graph = graph
        self.community_target = community_target
        # self.node_target = node_target
        # Compute the number of nodes in a community C that are in the same connected component of u
        self.V_u_C = self.num_nodes_in_same_component()
        # Get the number of intra-community edges for u.
        self.E_u_C = self.get_intra_comminty_edges()
        # Get the number of inter-community edges for u.
        self.E_u_C_bar = self.get_inter_community_edges()
    
    def community_hiding(self, community_target, edge_budget: int) -> nx.Graph:
        """
        Hide the target community using the safeness metric.

        Parameters
        ----------
        community_target : _type_
            Community to hide.
        edge_budget : int
            Budget of edges to use.

        Returns
        -------
        nx.Graph
            Graph with the target community hidden.
        """
        initial_budget = edge_budget
        while True:
            n_p = self.get_node_minimum_add_ratio(community_target)
            n_t = self.find_external_node(n_p, community_target)
            eps_add = self.get_addition_gain((n_p, n_t), community_target)
            
            n_k, n_l = self.get_best_del_excl_bridges(community_target)
            if n_k == None and n_l == None:
                eps_del = -1
            else:
                eps_del = self.get_deletion_gain((n_k, n_l), community_target)
            
            if eps_add >= eps_del:
                self.graph.add_edge(n_p, n_t)
            elif eps_del > 0:
                self.graph.remove_edge(n_k, n_l)
            
            edge_budget -= 1
            
            if edge_budget <= 0 or (eps_add <= 0 and eps_del <= 0):
                break
        steps = initial_budget - edge_budget
        return self.graph, steps
    
    def get_node_minimum_add_ratio(self, community_target: List[int])->int:
        """
        Computes for each node n inside the target community, the fraction of
        n’s edges that point outside C.

        Parameters
        ----------
        community_target : List[int]
            Target community.

        Returns
        -------
        min_add_ratio : int
            Node with the minimum add ratio.
        """
        # List of Tuple of (node, min_add_ratio)
        node_min_add_ratio = list()
        for n in community_target:
            min_add_ratio = 0
            for neighbor in self.graph.neighbors(n):
                if neighbor not in community_target:
                    min_add_ratio += 1
            min_add_ratio = min_add_ratio / self.graph.degree(n)
            node_min_add_ratio.append((n, min_add_ratio))
        
        # Get the node with the minimum add ratio
        min_add_ratio = min(node_min_add_ratio, key=lambda x: x[1])
        return min_add_ratio[0]
    
    def find_external_node(self, n_p: int, community_target: List[int]) -> int:
        """
        Find a node n_t not in community_target, such that the edge (n_p, n_t)
        does not exist.
        
        Parameters
        ----------
        n_p : int
            Node p.
        community_target : List[int]
            Target community.

        Returns
        -------
        int
            Destination node.
        """
        # get neighbors of n_p
        neighbors_p = self.graph.neighbors(n_p)
        for n_t in self.graph.nodes():
            if n_t not in community_target and n_t not in neighbors_p:
                return n_t
    
    def get_best_del_excl_bridges(self, community_target: List[int]) -> Tuple[int, int]:
        """
        It works in two phases:
            1. It excludes bridge edges that, if deleted, could disconnect 
                target community.
            2. For each remeining edge, it computes the value specified in 
                theorem 8.

        Parameters
        ----------
        community_target : 
            Community target.

        Returns
        -------
        Tuple[int, int]
            Edge with the maximum eps_del.
        """
        graph = self.graph.copy()
        # Delete all bridge edges that, if deleted, could disconnect target community.
        for node in community_target:
            # Get the neighbors of the node
            neighbors = self.graph.neighbors(node)
            for neighbor in neighbors:
                if neighbor not in community_target:
                    # Check if the edge is a bridge
                    if self.is_bridge((node, neighbor)):
                        graph.remove_edge(node, neighbor)

        # List of Tuple of (edge, eps_del)
        esp_del_list = list()
        for node in community_target:
            # Get the neighbors of the node
            neighbors = self.graph.neighbors(node)
            for neighbor in neighbors:
                if neighbor not in community_target:
                    eps_del = self.get_deletion_gain((node, neighbor), community_target)
                    esp_del_list.append(((node, neighbor), eps_del))
        
        # Get the edge with the maximum eps_del
        if len(esp_del_list) < 1:
            return (None, None)
        max_eps_del = max(esp_del_list, key=lambda x: x[1])
        return max_eps_del[0]
    
    def is_bridge(self, edge: Tuple[int, int]) -> bool:
        """
        Check if the edge (node, neighbor) is a bridge, i.e. if we remove it 
        the graph will be disconnected.

        Parameters
        ----------
        edge : Tuple[int, int]
            Edge to check.

        Returns
        -------
        bool
            True if the edge is a bridge, False otherwise.
        """
        graph = self.graph.copy()
        graph.remove_edge(*edge)
        return not nx.is_connected(graph)
    
    def get_addition_gain(self, edge: Tuple[int, int], community_target: List[int])->float:
        """
        Computes the addition gain of adding an edge.

        Parameters
        ----------
        edge : Tuple[int, int]
            Edge to add.
        community_target : List[int]
            Community target.

        Returns
        -------
        float
            Addition gain.
        """
        graph = self.graph.copy()
        # Compute the safeness before and after adding the edge.
        safeness_before = self.compute_community_safeness(
            graph, community_target)
        graph.add_edge(*edge)
        safeness_after = self.compute_community_safeness(
            graph, community_target)
        return safeness_after - safeness_before
    
    def get_deletion_gain(self, edge: Tuple[int, int], community_target: List[int])->float:
        """
        Computes the deletion gain of deleting an edge.

        Parameters
        ----------
        edge : Tuple[int, int]
            Edge to delete.
        community_target : List[int]
            Community target.

        Returns
        -------
        float
            Delete gain.
        """
        graph = self.graph.copy()
        # Compute the safeness before and after adding the edge.
        safeness_before = self.compute_community_safeness(
            graph, community_target)
        graph.remove_edge(*edge)
        safeness_after = self.compute_community_safeness(
            graph, community_target)
        return safeness_after - safeness_before
    
    
    def compute_community_safeness(self, graph, community_target: List[int]) -> float:
        """
        Computes the community safeness of the community.
        
        Parameters
        ----------
        community_taget: List[int]
            The community that we want to compute the safeness.

        Returns
        -------
        float
            The community safeness.
        """
        community_safeness = 0
        for node in community_target:
            community_safeness += self.compute_node_safeness(
                graph, community_target, node)
        return community_safeness / len(community_target)
    
    def compute_node_safeness(
        self, 
        graph: nx.Graph, 
        community_target: List[int], 
        node: int) -> float:
        """
        Computes the node safeness of the node in the community.
        
        Parameters
        ----------
        graph: nx.Graph
            The graph.
        community_target: List[int]
            The community of the node that we want to compute the safeness.
        
        node: int
            The node that we want to compute the safeness.

        Returns
        -------
        sigma_u_C: float
            The node safeness.
        """
        # Get the degree of u.
        deg_u = graph.degree(node)

        # Compute the node safeness.
        assert len(community_target) > 1, "The community must have at least 2 nodes."
        assert deg_u > 0, "The node must have at least 1 edge."
        sigma_u_C = 0.5*((self.V_u_C[node] - len(self.E_u_C)) /
                         (len(community_target) - 1)) + 0.5*(len(self.E_u_C_bar) / deg_u)
        return sigma_u_C
    
    # TEST, check if it si correct
    def num_nodes_in_same_component(self):
        """
        Computes the number of nodes in a community C that are in the same 
        connected component of u.

        Returns
        -------
        _type_
            _description_
        """
        V_u_C = dict()
        # Create a subgraph induced by the nodes in community C
        subgraph = self.graph.subgraph(self.community_target)

        # Compute the connected components of the subgraph
        components = list(nx.connected_components(subgraph))
        # Find the component that contains node u
        for u in self.community_target:
            V_u_C[u] = 0
            for component in components:
                if u in component:
                    # Return the number of nodes in the component
                    V_u_C[u] = len(component)
        return V_u_C

    def get_intra_comminty_edges(self) -> List[int]:
        """
        Get the intra-community edges of the community.

        Returns
        -------
        intra_community_edges: Set[int]
            The intra-community edges of the community.
        """
        intra_community_edges = list()
        for u in self.community_target:
            for v in self.community_target:
                if u != v and self.graph.has_edge(u, v) and (v, u) not in intra_community_edges:
                    intra_community_edges.append((u, v))
        return intra_community_edges

    def get_inter_community_edges(self) -> List[int]:
        """
        Get the inter-community edges of the community.

        Returns
        -------
        inter_community_edges: Set[int]
            The inter-community edges of the community.
        """
        inter_community_edges = list()
        for u in self.community_target:
            for v in self.graph.neighbors(u):
                if v not in self.community_target:
                    inter_community_edges.append((u, v))
        return inter_community_edges

## Similarity Metrics

In [ ]:
class CommunitySimilarity():
    """Class to compute the similarity between two lists of integers"""
    def __init__(self, function_name: str) -> None:
        self.function_name = function_name

    def select_similarity_function(self) -> Callable:
        """
        Select the similarity function to use

        Returns
        -------
        Callable
            Similarity function to use
        """
        if self.function_name == SimilarityFunctionsNames.JAC.value:
            return self.jaccard_similarity
        elif self.function_name == SimilarityFunctionsNames.OVE.value:
            return self.overlap_similarity
        elif self.function_name == SimilarityFunctionsNames.SOR.value:
            return self.sorensen_similarity
        else:
            raise Exception("Similarity function not found")

    @staticmethod
    def jaccard_similarity(a: List[int], b: List[int]) -> float:
        """
        Compute the Jaccard similarity between two lists, A and B:
            J(A,B) = |A ∩ B| / |A U B|

        Parameters
        ----------
        a : List[int]
            First List
        b : List[int]
            Second List

        Returns
        -------
        float
            Jaccard similarity between the two lists, between 0 and 1
        """
        assert len(a) > 0 and len(b) > 0, "Lists must be not empty"
        # Convert lists to sets
        a_set = set(a)
        b_set = set(b)
        # Compute the intersection and union
        intersection = a_set.intersection(b_set)
        union = a_set.union(b_set)
        return len(intersection) / len(union)

    @staticmethod
    def overlap_similarity(a: List[int], b: List[int]) -> float:
        """
        Compute the Overlap similarity between two lists, A and B:
            O(A,B) = |A ∩ B| / min(|A|, |B|)

        Parameters
        ----------
        a : List[int]
            First List
        b : List[int]
            Fist List

        Returns
        -------
        float
            Overlap coefficient between the two lists, value between 0 and 1
        """
        assert len(a) > 0 and len(b) > 0, "Lists must be not empty"
        # Convert lists to sets
        a_set = set(a)
        b_set = set(b)
        # Compute the intersection
        intersection = a_set.intersection(b_set)
        return len(intersection) / min(len(a_set), len(b_set))

    @staticmethod
    def sorensen_similarity(a: List[int], b: List[int]) -> float:
        """
        Compute the Sorensen similarity between two lists, A and B:
            S(A,B) = 2 * |A ∩ B| / (|A| + |B|)

        Parameters
        ----------
        a : List[int]
            First List
        b : List[int]
            Second List

        Returns
        -------
        float
            Sorensen similarity between the two lists, between 0 and 1
        """
        assert len(a) > 0 and len(b) > 0, "Lists must be not empty"
        # Convert lists to sets
        a_set = set(a)
        b_set = set(b)
        # Compute the intersection
        intersection = a_set.intersection(b_set)
        return 2 * len(intersection) / (len(a_set) + len(b_set))


class GraphSimilarity():
    """Class to compute the similarity between two graphs"""
    def __init__(self, function_name: str) -> None:
        """
        Initialize the GraphSimilarity class

        Parameters
        ----------
        function_name : str
            Name of the similarity function to use
        """
        self.function_name = function_name

    def select_similarity_function(self) -> Callable:
        """
        Select the similarity function to use

        Returns
        -------
        Callable
            Similarity function to use
        """
        if self.function_name == SimilarityFunctionsNames.GED.value:
            return self.graph_edit_distance
        elif self.function_name == SimilarityFunctionsNames.JAC_1.value:
            return self.jaccard_similarity_1
        elif self.function_name == SimilarityFunctionsNames.JAC_2.value:
            return self.jaccard_similarity_2
        else:
            raise Exception("Similarity function not found")

    def graph_edit_distance(self, g: nx.Graph, h: nx.Graph) -> float:
        """
        Compute the graph edit distance between two graphs, then normalize it
        using a null graph:
            GED(G1,G2)/[GED(G1,G0) + GED(G2,G0)]  with G0 = null graph

        Parameters
        ----------
        g : nx.Graph
            First graph
        h : nx.Graph
            Second graph

        Returns
        -------
        graph_distance : float
            Graph edit distance between the two graphs normalized
        """
        # Slow, but precise
        # graph_distance = nx.graph_edit_distance(self.graph, self.old_graph)

        # Faster approximation of the graph edit distance
        graph_distance = next(nx.optimize_graph_edit_distance(g, h))
        # Normalize
        g_dist_1 = next(nx.optimize_graph_edit_distance(g, nx.null_graph()))
        g_dist_2 = next(nx.optimize_graph_edit_distance(h, nx.null_graph()))
        graph_distance /= (g_dist_1 + g_dist_2)
        return graph_distance

    def jaccard_similarity_1(self, g: nx.Graph, h: nx.Graph) -> float:
        """
        Compute the Jaccard Similarity between two graphs
        J(G, H) = (∑_{i,j} |A_{ij}^G - A_{i,j}^H|) / (∑_{i,j} max(A_{i,j)^G, A_{i,j}^H))

        Parameters
        ----------
        g : nx.Graph
            First graph
        h : nx.Graph
            Second graph

        Returns
        -------
        jaccard_sim : float
            Jaccard Similarity between the two graphs, between 0 and 1,
            where 0 means the two graphs are identical and 1 means they are
            completely different
        """
        # Get adjacency matrices
        g_matrix = nx.to_numpy_array(g)
        h_matrix = nx.to_numpy_array(h)
        # Ensure G and H have the same shape
        if g_matrix.shape != h_matrix.shape:
            raise ValueError("Input matrices must have the same shape.")
        # Calculate the numerator (sum of absolute differences)
        numerator = np.sum(np.abs(g_matrix - h_matrix))
        # Calculate the denominator (sum of element-wise maximum values)
        denominator = np.sum(np.maximum(g_matrix, h_matrix))
        # Calculate the Jaccard similarity
        jaccard_sim = numerator / denominator
        return jaccard_sim

    def jaccard_similarity_2(self, g: nx.Graph, h: nx.Graph) -> float:
        """
        Compute the Jaccard Similarity between two graphs, second version

        Parameters
        ----------
        g : nx.Graph
            First graph
        h : nx.Graph
            Second graph

        Returns
        -------
        float
            jaccard similarity between the two graphs
        """
        g = g.edges()
        h = h.edges()
        i = set(g).intersection(h)
        j = round(len(i) / (len(g) + len(h) - len(i)), 3)
        # Normalize to have 0 if the graphs are identical and 1 if they are
        # completely different
        return 1-j


## Enviroment

In [ ]:
class GraphEnvironment(object):
    """Enviroment where the agent will act, it will be a graph with a community"""

    def __init__(
        self,
        graph_path: str = HyperParams.GRAPH_NAME.value,
        community_detection_algorithm: str = HyperParams.DETECTION_ALG_NAME.value,
        beta: float = HyperParams.BETA.value,
        tau: float = HyperParams.TAU.value,
        community_similarity_function: str = SimilarityFunctionsNames.SOR.value,
        graph_similarity_function: str = SimilarityFunctionsNames.JAC_1.value,
    ) -> None:
        """Constructor for Graph Environment
        Parameters
        ----------
        graph_path : str, optional
            Path of the graph to load, by default HyperParams.GRAPH_NAME.value
        community_detection_algorithm : str
            Name of the community detection algorithm to use
        beta : float, optional
            Percentage of edges to remove, by default HyperParams.BETA.value
        tau : float, optional
            Strength of the deception constraint, value between 0 and 1, with 1
            we have a soft constraint, hard constraint otherwise, by default
            HyperParams.T.value
        community_similarity_function : str, optional
            Name of the community similarity function to use, by default
            SimilarityFunctionsNames.SOR.value
        graph_similarity_function : str, optional
            Name of the graph similarity function to use, by default
            SimilarityFunctionsNames.JAC_1.value
        """
        random.seed(time.time())
        self.device = torch.device(
            'cuda:0' if torch.cuda.is_available() else 'cpu')
        # ° ---- GRAPH ---- ° #
        # Load the graph from the dataset folder
        if graph_path is None:
            # Generate a synthetic graph
            self.graph, graph_path = Utils.generate_lfr_benchmark_graph()
        else:
            self.graph = Utils.import_mtx_graph(graph_path)
        
        # Build node features using Node2Vec, set the embedding dimension to 128.
        self.embedding_model = Node2Vec(
            dimensions=HyperParams.EMBEDDING_DIM.value)
        print("* * Compute Node Embedding using Node2Vec for nodes features")
        print("* * ...")
        self.embedding_model.fit(self.graph)
        print("* * End Embedding Computation")
        self.embedding = self.embedding_model.get_embedding()
        # Add the embedding to the graph
        for node in self.graph.nodes():
            self.graph.nodes[node]["x"] = torch.tensor(self.embedding[node])
            # delete all the other features
        
        # Delete edges without "weight" attribute
        for edge in self.graph.edges():
            if "weight" not in self.graph.edges[edge]:
                self.graph.remove_edge(*edge)
        
        # Save the original graph to restart the rewiring process at each episode
        self.original_graph = self.graph.copy()
        # Save the graph state before the action, used to compute the metrics
        self.old_graph = None
        # Get the Number of connected components
        self.n_connected_components = nx.number_connected_components(
            self.graph)

        # ° ---- HYPERPARAMETERS ---- ° #
        assert beta >= 0 and beta <= 100, "Beta must be between 0 and 100"
        assert tau >= 0 and tau <= 1, "T value must be between 0 and 1"
        # Percentage of edges to remove
        self.beta = beta
        self.tau = tau
        # Weights for the reward and the penalty
        self.lambda_metric = None  # lambda_metric
        self.alpha_metric = None  # alpha_metric

        # ° ---- SIMILARITY FUNCTIONS ---- ° #
        # Select the similarity function to use to compare the communities
        self.community_similarity = CommunitySimilarity(
            community_similarity_function).select_similarity_function()
        self.graph_similarity = GraphSimilarity(
            graph_similarity_function).select_similarity_function()

        # ° ---- COMMUNITY DETECTION ---- ° #
        # Name of the environment and the community detection algorithm
        self.env_name = graph_path.split("/")[-1].split(".")[0]
        self.detection_alg = community_detection_algorithm
        # Community Algorithms objects
        self.detection = CommunityDetectionAlgorithm(
            community_detection_algorithm)
        # Metrics
        self.old_penalty_value = 0
        # Compute the community structure of the graph, before the action,
        # i.e. before the deception
        self.original_community_structure = self.detection.compute_community(
            self.graph)
        # ! It is a NodeClustering object
        self.old_community_structure = self.original_community_structure
        self.new_community_structure = None

        # ° ---- COMMUNITY DECEPTION ---- ° #
        # Choose one of the communities found by the algorithm, as initial
        # community we choose the community with the highest number of nodes
        self.community_target = max(
            self.original_community_structure.communities, key=len)
        if len(self.community_target) <= 1:
            raise Exception("Community target must have at least two node.")

        # Choose a node randomly from the community, as initial node to remove
        self.node_target = random.choice(self.community_target)

        # ° ---- REWIRING STEP ---- ° #
        # Compute the edge budget for the graph, i.e. the mean degree of the 
        # graph times the parameter beta
        self.edge_budget = self.get_edge_budget() #* self.beta
        # Amount of budget used
        self.used_edge_budget = 0
        # Max Rewiring Steps during an episode, set a limit to avoid infinite 
        # episodes in case the agent does not find the target node
        self.max_steps = self.edge_budget #* HyperParams.MAX_STEPS_MUL.value
        # Whether the budget for the graph rewiring is exhausted, or the target
        # node does not belong to the community anymore
        self.stop_episode = False
        self.rewards = 0
        # Reward of the previous step
        self.old_rewards = 0
        # Compute the set of possible actions
        self.possible_actions = self.get_possible_actions()
        # Length of the list of possible actions to add
        self.len_add_actions = len(self.possible_actions["ADD"])

        # ° ---- PRINT ENVIRONMENT INFO ---- ° #
        # Print the environment information
        self.print_env_info()

    ############################################################################
    #                       GETTERS FUNCTIONS                                  #
    ############################################################################

    def get_edge_budget(self) -> int:
        """
        Computes the edge budget for each graph

        Returns
        -------
        int
            Edge budgets of the graph
        """
        # Get the mean degree of the graph
        # return int(self.graph.number_of_edges() / self.graph.number_of_nodes())
        # Number of edges % beta
        # return int(math.ceil((self.graph.number_of_edges() * self.beta / 100)))
        # Nember of edges % beta / lenght of community
        return int(math.ceil((self.graph.number_of_edges() * self.beta / 100)) / len(self.community_target))

    def get_penalty(self) -> float:
        """
        Compute the metrics and return the penalty to subtract from the reward

        Returns
        -------
        penalty: float
            Penalty to subtract from the reward
        """
        # ° ---- COMMUNITY DISTANCE ---- ° #
        community_distance = self.new_community_structure.normalized_mutual_information(
            self.old_community_structure).score
        # In NMI 1 means that the two community structures are identical,
        # 0 means that they are completely different
        # We want to maximize the NMI, so we subtract it from 1
        community_distance = 1 - community_distance
        # ° ---- GRAPH DISTANCE ---- ° #
        graph_distance = self.graph_similarity(self.graph, self.old_graph)
        # ° ---- PENALTY ---- ° #
        assert self.alpha_metric is not None, "Alpha metric is None, must be set in grid search"
        penalty = self.alpha_metric * community_distance + \
            (1 - self.alpha_metric) * graph_distance
        # Subtract the metric value of the previous step
        penalty -= self.old_penalty_value
        # Update with the new values
        self.old_penalty_value = penalty
        return penalty

    def get_reward(self) -> Tuple[float, bool]:
        """
        Computes the reward for the agent, it is a 0-1 value function, if the
        target node still belongs to the community, the reward is 0 minus the
        penalty, otherwise the reward is 1 minus the penalty.

        As new community target after the action, we consider the community
        that contains the target node, if this community satisfies the deception
        constraint, the episode is finished, otherwise not.

        Returns
        -------
        reward : float
            Reward of the agent
        done : bool
            Whether the episode is finished, if the target node does not belong
            to the community anymore, the episode is finished
        """
        assert self.lambda_metric is not None, "Lambda metric is None, must be set in grid search"
        # Get the target community in the new community structure that
        # contains the target node
        for community in self.new_community_structure.communities:
            if self.node_target in community:
                new_community_target = community
                break
        assert new_community_target is not None, "New community target is None"
        # ° ---------- PENALTY ---------- ° #
        # Compute the metric to subtract from the reward
        penalty = self.get_penalty()
        # If the target node does not belong to the community anymore,
        # the episode is finished
        if len(new_community_target) == 1:
            reward = 1 - (self.lambda_metric * penalty)
            return reward, True
        # ° ---- COMMUNITY SIMILARITY ---- ° #
        # Remove target node from the communities, but first copy the lists
        # to avoid modifying them
        new_community_target_copy = new_community_target.copy()
        new_community_target_copy.remove(self.node_target)
        community_target_copy = self.community_target.copy()
        community_target_copy.remove(self.node_target)
        # Compute the similarity between the new communities
        community_similarity = self.community_similarity(
            new_community_target_copy,
            community_target_copy,
        )
        # Delete the copies
        del new_community_target_copy, community_target_copy
        # ° ---------- REWARD ---------- ° #
        if community_similarity <= self.tau:
            # We have reached the deception constraint, the episode is finished
            reward = 1 - (self.lambda_metric * penalty)
            return reward, True
        reward = 0 - (self.lambda_metric * penalty)
        return reward, False

    def get_possible_actions(self) -> dict:
        """
        Returns all the possible actions that can be applied to the graph
        given a source node (self.node_target). The possible actions are:
            - Add an edge between the source node and a node outside the community
            - Remove an edge between the source node and a node inside the community

        Returns
        -------
        self.possible_actions : dict
            Dictionary containing the possible actions that can be applied to
            the graph. The dictionary has two keys: "ADD" and "REMOVE", each
            key has a list of tuples as value, where each tuple is an action.
        """
        possible_actions = {"ADD": set(), "REMOVE": set()}
        # Helper functions to check if a node is in/out-side the community

        def in_community(node):
            return node in self.community_target

        def out_community(node):
            return node not in self.community_target

        u = self.node_target
        for v in self.graph.nodes():
            if u == v:
                continue
            # We can remove an edge iff both nodes are in the community
            if in_community(u) and in_community(v):
                if self.graph.has_edge(u, v):
                    if (v, u) not in possible_actions["REMOVE"]:
                        possible_actions["REMOVE"].add((u, v))
            # We can add an edge iff one node is in the community and the other is not
            elif (in_community(u) and out_community(v)) \
                    or (out_community(u) and in_community(v)):
                # Check if there is already an edge between the two nodes
                if not self.graph.has_edge(u, v):
                    if (v, u) not in possible_actions["ADD"]:
                        possible_actions["ADD"].add((u, v))
        return possible_actions

    ############################################################################
    #                       EPISODE RESET FUNCTIONS                            #
    ############################################################################

    def reset(self, graph_reset=True) -> nx.Graph:
        """
        Reset the environment
        
        Parameters
        ----------
        graph_reset : bool, optional
            Whether to reset the graph to the original state, by default True

        Returns
        -------
        self.graph : nx.Graph
            Graph state after the reset, i.e. the original graph
        """
        self.used_edge_budget = 0
        self.stop_episode = False
        self.rewards = 0
        self.old_rewards = 0
        if graph_reset:
            self.graph = self.original_graph.copy()
        self.old_graph = None
        self.old_penalty_value = 0
        self.old_community_structure = self.original_community_structure
        self.possible_actions = self.get_possible_actions()
        return self.graph

    def change_target_node(self, node_target: int = None) -> None:
        """
        Change the target node to remove from the community

        Parameters
        ----------
        node_target : int, optional
            Node to remove from the community, by default None
        """
        if node_target is None:
            # Choose a node randomly from the community
            old_node = self.node_target
            while self.node_target == old_node:
                random.seed(time.time())
                self.node_target = random.choice(self.community_target)
        else:
            self.node_target = node_target

    def change_target_community(
            self,
            community: List[int] = None,
            node_target: int = None) -> None:
        """
        Change the target community from which we want to hide the node

        Parameters
        ----------
        community : List[int]
            Community of node we want to remove from it
        node_target : int
            Node to remove from the community
        """
        if community is None:
            # Select randomly a new community target different from the last one
            old_community = self.community_target.copy()
            done = False
            while not done:
                random.seed(time.time())
                self.community_target = random.choice(
                    self.original_community_structure.communities)
                # Check condition on new community
                if (len(self.community_target) > 1 and \
                        self.community_target != old_community) or \
                            len(self.original_community_structure.communities) < 2:
                    done = True
            del old_community
        else:
            self.community_target = community
        # Change the target node to remove from the community
        self.change_target_node(node_target=node_target)

    ############################################################################
    #                      EPISODE STEP FUNCTIONS                              #
    ############################################################################
    def step(self, action: int) -> Tuple[nx.Graph, float, bool, bool]:
        """
        Step function for the environment

        Parameters
        ----------
        action : int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action (out source node is always the target node).

        Returns
        -------
        self.graph : nx.Graph
            Graph state after the action
        self.rewards : float
            Reward of the agent
        self.stop_episode : bool
            If the budget for the graph rewiring is exhausted, or the target
            node does not belong to the community anymore, the episode is finished
        done : bool
            Whether the episode is finished, if the target node does not belong
            to the community anymore, the episode is finished.
        """
        # ° ---- ACTION ---- ° #
        # Save the graph state before the action, used to compute the metrics
        self.old_graph = self.graph.copy()
        # Take action, add/remove the edge between target node and the model output
        budget_consumed = self.apply_action(action)
        # Set a negative reward if the action has not been applied
        if budget_consumed == 0:
            self.rewards = -1
            # The state is the same as before
            # return self.data_pyg, self.rewards, self.stop_episode
            return self.graph, self.rewards, self.stop_episode, False

        # ° ---- COMMUNITY DETECTION ---- ° #
        # Compute the community structure of the graph after the action
        self.new_community_structure = self.detection.compute_community(
            self.graph)

        # ° ---- REWARD ---- ° #
        self.rewards, done = self.get_reward()
        # If the target node does not belong to the community anymore,
        # the episode is finished
        if done:
            self.stop_episode = True

        # ° ---- BUDGET ---- ° #
        # Compute used budget
        self.used_edge_budget += budget_consumed
        # If the budget for the graph rewiring is exhausted, stop the episode
        if self.edge_budget - self.used_edge_budget < 1:
            self.stop_episode = True
            # If the budget is exhausted, and the target node still belongs to
            # the community, the reward is negative
            # if not done:
            #    self.rewards = -2

        self.old_community_structure = self.new_community_structure
        return self.graph, self.rewards, self.stop_episode, done

    def apply_action(self, action: int) -> int:
        """
        Applies the action to the graph, if there is an edge between the two
        nodes, it removes it, otherwise it adds it

        Parameters
        ----------
        action : int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action (out source node is always the target node).

        Returns
        -------
        budget_consumed : int
            Amount of budget consumed, 1 if the action has been applied, 0 otherwise
        """
        action = (self.node_target, action)
        # We need to take into account both the actions (u,v) and (v,u)
        action_reversed = (action[1], action[0])
        if action in self.possible_actions["ADD"]:
            self.graph.add_edge(*action, weight=1)
            self.possible_actions["ADD"].remove(action)
            return 1
        elif action_reversed in self.possible_actions["ADD"]:
            self.graph.add_edge(*action_reversed, weight=1)
            self.possible_actions["ADD"].remove(action_reversed)
            return 1
        elif action in self.possible_actions["REMOVE"]:
            self.graph.remove_edge(*action)
            self.possible_actions["REMOVE"].remove(action)
            return 1
        elif action_reversed in self.possible_actions["REMOVE"]:
            self.graph.remove_edge(*action_reversed)
            self.possible_actions["REMOVE"].remove(action_reversed)
            return 1
        return 0

    ############################################################################
    #                           ENVIRONMENT INFO                               #
    ############################################################################
    def print_env_info(self) -> None:
        """Print the environment information"""
        print("*"*20, "Environment Information", "*"*20)
        print("* Graph Name:", self.env_name)
        print("*", self.graph)
        print("* Community Detection Algorithm:", self.detection_alg)
        print("* Number of communities found:",
              len(self.original_community_structure.communities))
        # print("* Rewiring Budget:", self.edge_budget, "=", self.beta, "*", self.graph.number_of_edges(), "/ 100",)
        print("* Rewiring Budget: (n_edges/n_nodes)*BETA =",
              self.graph.number_of_edges(), "/",
              self.graph.number_of_nodes(), "*", self.beta, "=",
              int(self.graph.number_of_edges() / self.graph.number_of_nodes())*self.beta)
        print("* Weight of the Deception Constraint:", self.tau)
        print("*", "-"*58, "\n")

## Agent

In [ ]:
class Agent:
    def __init__(
            self,
            env: GraphEnvironment,
            state_dim: int = HyperParams.EMBEDDING_DIM.value,
            hidden_size_1: int = HyperParams.HIDDEN_SIZE_1.value,
            hidden_size_2: int = HyperParams.HIDDEN_SIZE_2.value,
            lr: List[float] = HyperParams.LR.value,
            gamma: List[float] = HyperParams.GAMMA.value,
            lambda_metrics: List[float] = HyperParams.LAMBDA.value,
            alpha_metrics: List[float] = HyperParams.ALPHA.value,
            eps: float = HyperParams.EPS_CLIP.value,
            best_reward: float = HyperParams.BEST_REWARD.value):
        """
        Initialize the agent.

        Parameters
        ----------
        env : GraphEnvironment
            Environment to train the agent on
        state_dim : int
            Dimensions of the state, i.e. length of the feature vector
        hidden_size_1 : int
            First A2C hidden layer size
        hidden_size_2 : int
            Second A2C hidden layer size
        action_dim : int
            Dimensions of the action (it is set to 1, to return a tensor N*1)
        lr : List[float]
            List of Learning rate, each element of the list is a learning rate
        gamma : List[float]
            List of gamma parameter, each element of the list is a gamma
        lambda_metrics : List[float]
            List of lambda parameter, each element of the list is a lambda used
            to balance the reward and the penalty
        alpha_metrics : List[float]
            List of alpha parameter, each element of the list is a alpha used
            to balance the two penalties
        eps : List[float]
            Value for clipping the loss function, each element of the list is a
            clipping value
        best_reward : float, optional
            Best reward, by default 0.8
        """
        # ° ----- Environment ----- ° #
        self.env = env

        # ° ----- A2C ----- ° #
        self.state_dim = state_dim # self.env.graph.number_of_nodes()
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.action_dim = self.env.graph.number_of_nodes()
        self.policy = ActorCritic(
            state_dim=self.state_dim,
            hidden_size_1=self.hidden_size_1,
            hidden_size_2=self.hidden_size_2,
            action_dim=self.action_dim,
            graph=self.env.graph
        )
        # Set device
        self.device = torch.device(
            'cuda:0' if torch.cuda.is_available() else 'cpu')
        # Move model to device
        self.policy.to(self.device)

        # ° ----- Hyperparameters ----- ° #
        # A2C hyperparameters
        self.lr_list = lr
        self.gamma_list = gamma
        self.eps = eps
        self.best_reward = best_reward
        # Environment hyperparameters
        self.lambda_metrics = lambda_metrics
        self.alpha_metrics = alpha_metrics
        # Hyperparameters to be set during grid search
        self.lr = None
        self.gamma = None
        self.alpha_metric = None
        self. optimizers = dict()

        # ° ----- Training ----- ° #
        # State, nx.Graph
        self.obs = None
        # Cumulative reward of the episode
        self.episode_reward = 0
        # Boolean variable to check if the episode is ended
        self.done = False
        # Boolean variable to check if the goal is reached
        self.goal = False
        # Number of steps in the episode
        self.step = 0
        # Tuple to store the values for each action
        self.SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
        self.saved_actions = []
        self.rewards = []
        # List of rewards for one episode
        self.episode_rewards = []
        # Initialize lists for logging, it contains: avg_reward, avg_steps per episode
        self.log_dict = HyperParams.LOG_DICT.value
        # Print agent info
        self.print_agent_info()

        # ° ----- Evaluation ----- ° #
        # List of actions performed during the evaluation
        self.action_list = {"ADD": [], "REMOVE": []}

    ############################################################################
    #                       PRE-TRAINING/TESTING                               #
    ############################################################################
    def reset_hyperparams(
            self,
            lr: float,
            gamma: float,
            lambda_metric: float,
            alpha_metric: float,
            test: bool = False) -> None:
        """
        Reset hyperparameters
        
        Parameters
        ----------
        lr : float
            Learning rate
        gamma : float
            Discount factor
        lambda_metric : float
            Lambda parameter used to balance the reward and the penalty
        alpha_metric : float
            Alpha parameter used to balance the two penalties
        test : bool, optional
            Print hyperparameters during training, by default False
        """
        # Set A2C hyperparameters
        self.lr = lr
        self.gamma = gamma
        # Set environment hyperparameters
        self.env.lambda_metric = lambda_metric
        self.env.alpha_metric = alpha_metric
        # Print hyperparameters if we are not testing
        if not test:
            self.print_hyperparams()
        # Clear logs, except for the training episodes
        for key in self.log_dict.keys():
            if key != 'train_episodes':
                self.log_dict[key] = list()
        # Clear action list
        self.saved_actions = []
        self.rewards = []
        self.episode_rewards = []
        # Clear state
        self.obs = None
        self.episode_reward = 0
        self.done = False
        self.goal = False
        self.step = 0
        self.optimizers = dict()

    def configure_optimizers(self) -> None:
        """
        Configure optimizers
        
        Returns
        -------
        optimizers : dict
            Dictionary of optimizers
        """
        actor_params = list(self.policy.actor.parameters())
        critic_params = list(self.policy.critic.parameters())
        self.optimizers['a_optimizer'] = torch.optim.Adam(
            actor_params, lr=self.lr)
        self.optimizers['c_optimizer'] = torch.optim.Adam(
            critic_params, lr=self.lr)

    ############################################################################
    #                            GRID SEARCH                                   #
    ############################################################################
    def grid_search(self) -> None:
        """Perform grid search on the hyperparameters"""
        for lr in self.lr_list:
            for gamma in self.gamma_list:
                for lambda_metric in self.lambda_metrics:
                    for alpha_metric in self.alpha_metrics:
                        # Change Hyperparameters
                        self.reset_hyperparams(
                            lr, gamma, lambda_metric, alpha_metric)
                        # Configure optimizers with the current learning rate
                        self.configure_optimizers()
                        # Training
                        log = self.training()
                        # Save results in correct folder
                        self.save_plots(log, self.get_path())
                        # Free memory
                        gc.collect()

    ############################################################################
    #                               TRAINING                                   #
    ############################################################################
    def training(self) -> dict:
        """
        Train the agent on the environment, change the target node every 10
        episodes and the target community every 100 episodes. The episode ends
        when the target node is isolated from the target community, or when the
        maximum number of steps is reached.
            
        Returns
        -------
        log_dict : dict
            Dictionary containing the training logs
        """
        episode = self.log_dict['train_episodes']
        epochs = trange(episode)  # epoch iterator
        self.policy.train()  # set model in train mode
        for i_episode in epochs:
            self.env.change_target_community()

            # Reset environment, original graph, and new set of possible actions
            self.obs = self.env.reset()
            self.episode_reward = 0
            self.done = False
            self.goal = False
            self.episode_rewards = []
            self.step = 0
            # Rewiring the graph until the target node is isolated from the
            # target community
            while not self.done and self.step < self.env.max_steps:
                self.rewiring()
            # perform on-policy backpropagation
            self.a_loss, self.v_loss = self.training_step()
            # Checkpoint best performing model
            if self.episode_reward / self.step >= self.best_reward:
                self.save_checkpoint()
                self.best_reward = self.episode_reward

            # ° Log
            # Get the list of reward of the last self.step steps
            rewards = self.episode_rewards[-self.step:]
            # If the goal is reached, multiply the last reward by 10
            if self.goal:
                rewards[-1] *= 10
            self.log_dict['train_reward_list'].append(rewards)
            self.log_dict['train_reward_mul'].append(sum(rewards)/len(rewards))

            self.log_dict['train_reward'].append(self.episode_reward)
            self.log_dict['train_steps'].append(self.step)
            self.log_dict['train_avg_reward'].append(
                self.episode_reward/self.step)
            self.log_dict['a_loss'].append(self.a_loss)
            self.log_dict['v_loss'].append(self.v_loss)

            # Send current statistics to screen
            epochs.set_description(
                f"* Episode {i_episode+1} " +
                f"| Mul Reward: {sum(rewards)/len(rewards):.2f}"
                f"| Avg Reward: {self.episode_reward/self.step:.2f} " +
                f"| Steps: {self.step} " +
                f"| Actor Loss: {self.a_loss:.2f} " +
                f"| Critic Loss: {self.v_loss:.2f}")
            del rewards
        return self.log_dict

    def rewiring(self, test=False) -> None:
        """
        Rewiring step, select action and take step in environment.
        
        Parameters
        ----------
        test : bool, optional
            If True, print rewiring action, by default False
        """
        # Select action: return a list of the probabilities of each action
        action_rl = self.select_action(self.obs)
        torch.cuda.empty_cache()
        # Save rewiring action if we are testing
        if test:
            edge = (self.env.node_target, action_rl)
            if edge in self.env.possible_actions["ADD"]:
                if not self.env.graph.has_edge(*edge):
                    self.action_list["ADD"].append(edge)
            elif edge in self.env.possible_actions["REMOVE"]:
                if self.env.graph.has_edge(*edge):
                    self.action_list["REMOVE"].append(edge)
        # Take action in environment
        self.obs, reward, self.done, self.goal = self.env.step(action_rl)

        # Update ra_losseward
        self.episode_reward += reward
        # Store the transition in memory, used for the training step
        self.rewards.append(reward)
        # Used for logging
        self.episode_rewards.append(reward)
        self.step += 1
        # print("STEP", self.step, "  GOAL:", self.goal, "  DONE:", self.done, "  REWARD:", reward)

    def select_action(self, state: Data) -> int:
        """
        Select action, given a state, using the policy network.
        
        Parameters
        ----------
        state : Data
            Graph state
        
        Returns
        -------
        action: int
            Integer representing a node in the graph, it will be the destination
            node of the rewiring action
        """
        concentration, value = self.policy(state)
        dist = torch.distributions.Categorical(concentration)
        action = dist.sample()
        self.saved_actions.append(
            self.SavedAction(dist.log_prob(action), value))
        return int(action.item())

    def training_step(self) -> Tuple[float, float]:
        """
        Perform a single training step of the A2C algorithm, which involves
        computing the actor and critic losses, taking gradient steps, and 
        resetting the rewards and action buffer.
        
        Returns
        -------
        mean_a_loss : float
            Mean actor loss
        mean_v_loss : float
            Mean critic loss
        """
        R = 0
        saved_actions = self.saved_actions
        policy_losses = []  # list to save actor (policy) loss
        value_losses = []  # list to save critic (value) loss
        returns = []  # list to save the true values
        # Compute the true value using rewards returned from the environment
        for r in self.rewards[::-1]:
            # calculate the discounted value
            R = r + self.gamma * R
            # insert to the beginning of the list
            returns.insert(0, R)
        # Normalize returns by subtracting mean and dividing by standard deviation
        # NOTE: May cause NaN problem
        if len(returns) > 1:
            returns = torch.tensor(returns)
            returns = (returns - returns.mean()) / (returns.std() + self.eps)
        else:
            returns = torch.tensor(returns)
        # Computing losses
        for (log_prob, value), R in zip(saved_actions, returns):
            # Difference between true value and estimated value from critic
            advantage = R - value.item()
            # calculate actor (policy) loss
            policy_losses.append(-log_prob * advantage)
            # calculate critic (value) loss using L1 smooth loss
            value_losses.append(F.smooth_l1_loss(
                value, torch.tensor([R]).to(self.device)))
        # take gradient steps
        self.optimizers['a_optimizer'].zero_grad()
        a_loss = torch.stack(policy_losses).sum()
        a_loss.backward()
        self.optimizers['a_optimizer'].step()
        self.optimizers['c_optimizer'].zero_grad()
        v_loss = torch.stack(value_losses).sum()
        v_loss.backward()
        self.optimizers['c_optimizer'].step()
        # Compute mean losses
        mean_a_loss = torch.stack(policy_losses).mean().item()
        mean_v_loss = torch.stack(value_losses).mean().item()
        # reset rewards and action buffer
        del self.rewards[:]
        del self.saved_actions[:]
        return mean_a_loss, mean_v_loss

    ############################################################################
    #                               TEST                                       #
    ############################################################################
    def test(
            self,
            lr: float,
            gamma: float,
            lambda_metric: float,
            alpha_metric: float,
            model_path: str,
            graph_reset=True) -> nx.Graph:
        """Hide a given node from a given community"""
        # Set hyperparameters to select the correct folder
        self.reset_hyperparams(lr, gamma, lambda_metric, alpha_metric, True)
        # Load best performing model
        self.load_checkpoint(path=model_path)
        # Set model in evaluation mode
        self.policy.eval()
        self.obs = self.env.reset(graph_reset)
        # Rewiring the graph until the target node is isolated from the
        # target community
        while not self.done and self.step < self.env.max_steps:
            self.rewiring(test=True)
        # if self.step >= self.env.max_steps:
        #     print("* !!!Maximum number of steps reached!!!")
        return self.obs

    ############################################################################
    #                            CHECKPOINTING                                 #
    ############################################################################
    def get_path(self) -> str:
        """
        Return the path of the folder where to save the plots and the logs
        
        Returns
        -------
        file_path : str
            Path to the correct folder
        """
        file_path = FilePaths.LOG_DIR.value + \
            f"{self.env.env_name}/{self.env.detection_alg}/" +\
            f"lr-{self.lr}/gamma-{self.gamma}/" +\
            f"lambda-{self.env.lambda_metric}/alpha-{self.env.alpha_metric}"
        return file_path

    def save_plots(self, log: dict, file_path: str) -> None:
        """
        Save training plots and logs

        Parameters
        ----------
        log : dict
            Dict containing the training logs
        file_path : str
            Path to the directory where to save the plots and the logs
        """
        Utils.check_dir(file_path)
        self.log(log)
        Utils.plot_training(
            log,
            self.env.env_name,
            self.env.detection_alg,
            file_path)

    def save_checkpoint(self):
        """Save checkpoint"""
        log_dir = self.get_path()
        # Check if the directory exists, otherwise create it
        Utils.check_dir(log_dir)
        checkpoint = dict()
        checkpoint['model'] = self.policy.state_dict()
        for key, value in self.optimizers.items():
            checkpoint[key] = value.state_dict()
        path = f'{log_dir}/model.pth'
        torch.save(checkpoint, path)

    def load_checkpoint(self, path=None):
        """Load checkpoint"""
        if path is None:
            log_dir = self.get_path()
            path = f'{log_dir}/model.pth'
        
        checkpoint = torch.load(path, map_location=self.device)
        self.policy.load_state_dict(checkpoint['model'])
        for key, _ in self.optimizers.items():
            self.optimizers[key].load_state_dict(checkpoint[key])

    def log(self, log_dict: dict):
        """Log data
        
        Parameters
        ----------
        log_dict : dict
            Dictionary containing the data to be logged
        """
        log_dir = self.get_path()
        Utils.check_dir(log_dir)
        file_name = f'{log_dir}/training_results.json'
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(log_dict, f, indent=4)

    ############################################################################
    #                   AGENT INFO AND PRINTING                                #
    ############################################################################
    def print_agent_info(self):
        # Print model architecture
        print("*", "-"*18, " Model Architecture ", "-"*18)
        # print("* Embedding dimension: ", self.state_dim)
        print("* Features vector size: ", self.state_dim)
        print("* A2C Hidden layer 1 size: ", self.hidden_size_1)
        print("* A2C Hidden layer 2 size: ", self.hidden_size_2)
        print("* Actor Action dimension: ", self.action_dim)
        print("*", "-"*58, "\n")
        # Print Hyperparameters List
        print("*", "-"*18, "Hyperparameters List", "-"*18)
        print("* Learning rate list: ", self.lr_list)
        print("* Gamma parameter list: ", self.gamma_list)
        print("* Lambda Metric list: ", self.lambda_metrics)
        print("* Alpha Metric list: ", self.alpha_metrics)
        print("*", "-"*58, "\n")

    def print_hyperparams(self):
        print("*", "-"*18, "Model Hyperparameters", "-"*18)
        print("* Learning rate: ", self.lr)
        print("* Gamma parameter: ", self.gamma)
        print("* Lambda Metric: ", self.env.lambda_metric)
        print("* Alpha Metric: ", self.env.alpha_metric)
        print("* Value for clipping the loss function: ", self.eps)


### A2C

In [ ]:

class ActorCritic(nn.Module):
    """ActorCritic Network"""

    def __init__(
        self, 
        state_dim: int, 
        hidden_size_1: int, 
        hidden_size_2: int, 
        action_dim: int,
        graph: nx.Graph):
        super(ActorCritic, self).__init__()
        self.actor = ActorNetwork(
            state_dim=state_dim,
            hidden_size_1=hidden_size_1,
            hidden_size_2=hidden_size_2,
            action_dim=action_dim
        )
        self.critic = CriticNetwork(
            state_dim=state_dim,
            hidden_size_1=hidden_size_1,
            hidden_size_2=hidden_size_2
        )
        self.device = torch.device(
            'cuda:0' if torch.cuda.is_available() else 'cpu')
        

    def forward(self, graph: nx.Graph, jitter=1e-20) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass, computes action and value

        Parameters
        ----------
        graph : nx.Graph
            Graph state
        jitter : float, optional
            Jitter value, by default 1e-20

        Returns
        -------
        Tuple[torch.Tensor, torch.Tensor]
            Tuple of concentration and value
        """
        # Convert graph to torch_geometric.data.Data
        state = from_networkx(graph).to(self.device)

        # Actor
        probs = self.actor(state)        
        # Use softplus to ensure concentration is positive, then add jitter to 
        # ensure numerical stability
        concentration = F.softplus(probs).reshape(-1) + jitter

        # Critic
        value = self.critic(state)
        return concentration, value


#### Encoder

In [ ]:
class GraphEncoder(nn.Module):
    def __init__(self, in_feature):
        super(GraphEncoder, self).__init__()

        self.in_feature = in_feature
        self.hidden_feature = 64
        self.out_feature = HyperParams.EMBEDDING_DIM.value


        self.conv1 = GCNConv(in_feature, self.hidden_feature)
        self.linear1 = nn.Linear(self.hidden_feature, self.out_feature)
        self.tanh = nn.Tanh()
        self.relu = torch.relu

    def forward(self,  Graph):
        x, edge_index, batch = Graph.x, Graph.edge_index, Graph.batch
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        embedding = global_mean_pool(x,batch)
        embedding = self.linear1(embedding)
        embedding = self.tanh(embedding)
        return embedding

#### Actor

In [ ]:
class ActorNetwork(nn.Module):
    """Actor Network"""

    def __init__(
            self,
            state_dim: int,
            hidden_size_1: int,
            hidden_size_2: int,
            action_dim: int):
        super(ActorNetwork, self).__init__()

        # self.graph_encoder = GraphEncoder(state_dim)
        self.conv1 = GCNConv(state_dim, hidden_size_1)
        
        self.lin1 = nn.Linear(hidden_size_1, hidden_size_1)
        self.lin2 = nn.Linear(hidden_size_1, hidden_size_2)
        # self.lin3 = nn.Linear(hidden_size_2, action_dim)
        self.lin3 = nn.Linear(hidden_size_2, 1)
        
        # self.relu = nn.LeakyReLU()
        self.relu = nn.ReLU()
        # self.tanh = nn.Tanh()
    
    def forward(self, data: torch.Tensor)->torch.Tensor:
        out = F.relu(self.conv1(data.x, data.edge_index))
        # out = out + data.x
        x = F.relu(self.lin1(out))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

#### Critic

In [ ]:

class CriticNetwork(nn.Module):
    def __init__(
            self,
            state_dim: int,
            hidden_size_1: int,
            hidden_size_2: int):
        super(CriticNetwork, self).__init__()

        # self.graph_encoder = GraphEncoder(state_dim)
        self.conv1 = GCNConv(state_dim, hidden_size_1)
        
        self.lin1 = nn.Linear(hidden_size_1, hidden_size_1)
        self.lin2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.lin3 = nn.Linear(hidden_size_2, 1)

        # self.relu = nn.LeakyReLU()
        self.relu = nn.ReLU()
        # self.relu = F.relu
        # self.tanh = nn.Tanh()

    
    def forward(self, data: torch.Tensor)->torch.Tensor:
        out = F.relu(self.conv1(data.x, data.edge_index))
        # x = out + data.x
        # x = torch.sum(x, dim=0)
        x = torch.sum(out, dim=0)
        # x = self.relu(self.lin1(data))
        x = self.relu(self.lin1(x))
        x = self.relu(self.lin2(x))
        x = self.lin3(x)
        return x


## Test

In [ ]:
def test(
    agent: Agent,
    beta: float,
    tau: float,
    model_path: str,
    eval_steps: int = HyperParams.STEPS_EVAL.value,
    lr: float = HyperParams.LR_EVAL.value,
    gamma: float = HyperParams.GAMMA_EVAL.value,
    lambda_metric: float = HyperParams.LAMBDA_EVAL.value,
    alpha_metric: float = HyperParams.ALPHA_EVAL.value)->None:
    """
    Function to evaluate the performance of the agent and compare it with 
    the baseline algorithms.
    
    The baseline algorithms are:
        - Random Hiding
        - Degree Hiding
        - Roam Heuristic

    Parameters
    ----------
    agent : Agent
        Agent to evaluate
    beta : float
        Beta parameter for the number of rewiring steps
    tau : float
        Tau parameter as constraint for community target similarity
    model_path : str
        Path to the model to load
    eval_steps : int, optional
        Number of episodes to test, by default 1000
    lr : float, optional
        Learning rate, by default 1e-3
    gamma : float, optional
        Discount factor, by default 0.3
    lambda_metric : float, optional
        Weight to balance the penalty and reward, by default 0.1
    alpha_metric : float, optional
        Weight to balance the penalties, by default 0.1
    """
    # Initialize the log dictionary
    
    log_dict = Utils.initialize_dict(HyperParams.ALGS_EVAL.value)
    
    # Set parameters in the environment
    agent.env.beta = beta
    agent.env.edge_budget = agent.env.get_edge_budget() * agent.env.beta
    agent.env.max_steps = agent.env.edge_budget * HyperParams.MAX_STEPS_MUL.value
    agent.env.tau = tau
    
    # Add environment parameters to the log dictionary
    log_dict["env"] = dict()
    log_dict["env"]["dataset"] = agent.env.env_name
    log_dict["env"]["detection_alg"] = agent.env.detection_alg
    log_dict["env"]["beta"] = beta
    log_dict["env"]["tau"] = tau
    log_dict["env"]["edge_budget"] = agent.env.edge_budget
    log_dict["env"]["max_steps"] = agent.env.max_steps
    
    # Add Agent Hyperparameters to the log dictionary
    log_dict["Agent"]["lr"] = lr
    log_dict["Agent"]["gamma"] = gamma
    log_dict["Agent"]["lambda_metric"] = lambda_metric
    log_dict["Agent"]["alpha_metric"] = alpha_metric
    
    # Start evaluation
    steps = trange(eval_steps, desc="Testing Episode")
    for step in steps:
        
        # Change the target community and node at each episode
        agent.env.change_target_community()
        
        # ° ------ Agent ------ ° #
        steps.set_description(f"* Testing Episode {step+1} | Agent Rewiring")
        start = time.time()
        new_graph = agent.test(
            lr=lr,
            gamma=gamma,
            lambda_metric=lambda_metric,
            alpha_metric= alpha_metric,
            model_path=model_path,
        )
        # "src/logs/lfr_benchmark_n-300/infomap/lr-0.0001/gamma-0.9/lambda-0.1/alpha-0.7"
        end = time.time() - start
        
        # ° Target node and community for this episode ° #
        # We set it after the test to change automatically at each episode
        community_structure = agent.env.original_community_structure
        community_target = agent.env.community_target
        node_target = agent.env.node_target
        # ° ------------------------------------------ ° #
        # Get new target community after deception
        agent_community = Utils.get_new_community(node_target, agent.env.new_community_structure)
        # Compute NMI between the new community structure and the original one
        agent_nmi = community_structure.normalized_mutual_information(
            agent.env.new_community_structure).score
        # Check if the goal of hiding the target node was achieved
        agent_goal = Utils.check_goal(agent.env, node_target, community_target, agent_community)
        # Save the metrics
        log_dict = save_metrics(
            log_dict, "Agent", agent_goal, agent_nmi, end, agent.step)

        
        # Perform Deception with the baseline algorithms
        # ° ------ Random Hiding ------ ° #
        steps.set_description(f"* Testing Episode {step+1} | Random Rewiring")
        random_hiding = RandomHiding(
            env=agent.env,
            steps=agent.env.edge_budget,
            target_community=community_target)

        start = time.time()
        rh_graph, rh_communities = random_hiding.hide_target_node_from_community()
        end = time.time() - start
        
        # Get new target community after deception
        rh_community = Utils.get_new_community(node_target, rh_communities)
        # Compute NMI between the new community structure and the original one
        rh_nmi = community_structure.normalized_mutual_information(
            rh_communities).score
        # Check if the goal of hiding the target node was achieved
        rh_goal = Utils.check_goal(
            agent.env, node_target, community_target, rh_community)
        # Save the metrics
        log_dict = save_metrics(
            log_dict, "Random", rh_goal, rh_nmi, end, agent.env.edge_budget-random_hiding.steps)
        

        # ° ------ Degree Hiding ------ ° #
        steps.set_description(f"* Testing Episode {step+1} | Degree Rewiring")
        degree_hiding = DegreeHiding(
            env=agent.env,
            steps=agent.env.edge_budget,
            target_community=community_target)

        start = time.time()
        dh_graph, dh_communities = degree_hiding.hide_target_node_from_community()
        end = time.time() - start
        
        # Get new target community after deception
        dh_community = Utils.get_new_community(node_target, dh_communities)
        # Compute NMI between the new community structure and the original one
        dh_nmi = community_structure.normalized_mutual_information(
            dh_communities).score
        # Check if the goal of hiding the target node was achieved
        dh_goal = Utils.check_goal(
            agent.env, node_target, community_target, dh_community)
        # Save the metrics
        log_dict = save_metrics(
            log_dict, "Degree", dh_goal, dh_nmi, end, agent.env.edge_budget-degree_hiding.steps)

        # ° ------ Roam Heuristic ------ ° #
        steps.set_description(f"* Testing Episode {step+1} | Roam Rewiring")
        # Apply Hide and Seek
        deception = RoamHiding(
            agent.env.original_graph.copy(), node_target, agent.env.detection_alg)
        start = time.time()
        di_graph, di_communities = deception.roam_heuristic(
            agent.env.edge_budget)
        end = time.time() - start
        
        # Get new target community after deception
        di_community = Utils.get_new_community(node_target, di_communities)
        # Compute NMI between the new community structure and the original one
        di_nmi = community_structure.normalized_mutual_information(
            di_communities).score
        # Check if the goal of hiding the target node was achieved
        di_goal = Utils.check_goal(
            agent.env, node_target, community_target, di_community)
        # Save the metrics
        log_dict = save_metrics(
            log_dict, "Roam", di_goal, di_nmi, end, agent.env.edge_budget)

        steps.set_description(f"* Testing Episode {step+1}")
    # Save the log
    path = FilePaths.TEST_DIR.value + \
        f"{log_dict['env']['dataset']}/{log_dict['env']['detection_alg']}/" + \
        f"tau-{tau}/beta-{beta}/" + \
        f"lr-{lr}/gamma-{gamma}/lambda-{lambda_metric}/alpha-{alpha_metric}/"
    Utils.check_dir(path)
    Utils.save_test(
        log_dict, 
        path, 
        "evaluation_node_hiding", 
        algs=["Agent", "Random", "Degree", "Roam"],
        metrics=["nmi", "goal", "time", "steps"])



################################################################################
#                               Utility Functions                              #
################################################################################
def save_metrics(
        log_dict: dict, alg: str, goal: int,
        nmi: float, time: float, steps: int) -> dict:
    """Save the metrics of the algorithm in the log dictionary"""
    log_dict[alg]["goal"].append(goal)
    log_dict[alg]["nmi"].append(nmi)
    log_dict[alg]["time"].append(time)
    log_dict[alg]["steps"].append(steps)
    return log_dict

In [ ]:
def community_deception(
    agent: Agent,
    community_target: List[int],
    beta: float,
    tau: float,
    model_path: str,
    eval_steps: int = HyperParams.STEPS_EVAL.value,
    lr: float = HyperParams.LR_EVAL.value,
    gamma: float = HyperParams.GAMMA_EVAL.value,
    lambda_metric: float = HyperParams.LAMBDA_EVAL.value,
    alpha_metric: float = HyperParams.ALPHA_EVAL.value)->None:
    
    # Compute the edge budget for the community hiding
    community_beta = 0.3
    # Number of edges % community BETA
    community_edge_budget = int(
        math.ceil((agent.env.graph.number_of_edges()*community_beta)))
    # Budget for each node
    node_edge_budget = int(math.ceil(community_edge_budget/len(community_target)))
        
    # Set parameters in the environment
    agent.env.beta = community_beta
    
    # agent.env.edge_budget = agent.env.get_edge_budget() * agent.env.beta
    agent.env.edge_budget = node_edge_budget
    
    agent.env.max_steps = agent.env.edge_budget * HyperParams.MAX_STEPS_MUL.value
    agent.env.tau = tau
    agent.env.community_target = community_target
    
    # Start evaluation
    steps = trange(eval_steps, desc="Testing Episode")

    log_dict = {"env": {}, "Agent": {}, "Safeness": {}}
    
    log_dict["env"]["dataset"] = agent.env.env_name
    log_dict["env"]["detection_alg"] = agent.env.detection_alg
    log_dict["env"]["beta"] = community_beta
    log_dict["env"]["tau"] = tau
    log_dict["env"]["edge_budget"] = agent.env.edge_budget
    log_dict["env"]["max_steps"] = agent.env.max_steps
    log_dict["env"]["n_nodes"] = agent.env.graph.number_of_nodes()
    log_dict["env"]["n_edges"] = agent.env.graph.number_of_edges()
    log_dict["env"]["n_nodes_community_target"] = len(community_target)
    
    log_dict["Agent"]["deception_score"] = []
    log_dict["Agent"]["nmi"] = []
    log_dict["Agent"]["goal"] = []
    log_dict["Agent"]["time"] = []
    log_dict["Agent"]["steps"] = []
    
    log_dict["Safeness"]["deception_score"] = []
    log_dict["Safeness"]["nmi"] = []
    log_dict["Safeness"]["goal"] = []
    log_dict["Safeness"]["time"] = []
    log_dict["Safeness"]["steps"] = []
    
    for step in steps:
        
        # Reset the environment
        agent.env.reset()
        
        orginal_communities = agent.env.original_community_structure
        deception_obj = DeceptionScore(community_target)
        
        # ° ------ Agent ------ ° #
        agent_goal_reached = False
        start = time.time()
        agent_steps = 0
        for node in community_target:
            agent.env.node_target = node
            # The agent possible action are changed in the test function, which
            # calls the reset function of the environment
            new_graph = agent.test(
                lr=lr,
                gamma=gamma,
                lambda_metric=lambda_metric,
                alpha_metric=alpha_metric,
                model_path=model_path,
                graph_reset=False,
            )
            agent_steps += agent.step
            if community_target not in agent.env.new_community_structure.communities:
                agent_goal_reached = True
                break
        end = time.time() - start
        log_dict["Agent"]["time"].append(end)
        log_dict["Agent"]["steps"].append(agent_steps)
        # Check if the goal of hiding the target community was achieved
        if agent_goal_reached:
            log_dict["Agent"]["goal"].append(1)
        else:
            log_dict["Agent"]["goal"].append(0)
        # Compute the deception score
        deception_score = deception_obj.compute_deception_score(
            agent.env.new_community_structure.communities,
            nx.number_connected_components(new_graph),
        )
        log_dict["Agent"]["deception_score"].append(deception_score)
        # Compute NMI between the new community structure and the original one
        nmi = orginal_communities.normalized_mutual_information(
            agent.env.new_community_structure)
        log_dict["Agent"]["nmi"].append(nmi.score)

        
        # ° ------ Safeness ------ ° #
        # Reset the environment
        agent.env.reset()
        start = time.time()
        safeness_obj = Safeness(
            agent.env.graph,
            community_target,
        )
        safeness_graph, steps = safeness_obj.community_hiding(
            community_target=community_target,
            edge_budget=agent.env.edge_budget*len(community_target)
        )
        end = time.time() - start
        log_dict["Safeness"]["time"].append(end)
        log_dict["Safeness"]["steps"].append(steps)
        # Compute the new community structure, after deception
        new_communities = agent.env.detection.compute_community(safeness_graph)
        safeness_deception_score = deception_obj.compute_deception_score(
            new_communities.communities,
            nx.number_connected_components(safeness_graph),
        )
        log_dict["Safeness"]["deception_score"].append(safeness_deception_score)
        safeness_nmi = orginal_communities.normalized_mutual_information(
            new_communities
        )
        log_dict["Safeness"]["nmi"].append(safeness_nmi.score)
        if community_target in new_communities.communities:
            log_dict["Safeness"]["goal"].append(0)
        else:
            log_dict["Safeness"]["goal"].append(1)
    
    # Save log_dict
    path = FilePaths.TEST_DIR.value + \
        f"{log_dict['env']['dataset']}/{log_dict['env']['detection_alg']}/" + \
        f"tau-{tau}/beta-{beta}/" + \
        f"lr-{lr}/gamma-{gamma}/lambda-{lambda_metric}/alpha-{alpha_metric}/"
    Utils.check_dir(path)
    Utils.save_test(
        log_dict, 
        path, 
        "evaluation_community_hiding", 
        algs=["Agent", "Safeness"],
        metrics=["deception_score", "nmi", "goal", "time", "steps"])

## Execution

In [ ]:
# NOTE To modify the hyperparameters, dataset, detection algorithm, etc. 
# NOTE  please refer to the file src/utils/utils.py in the class HyperParams
# ° --- Environment Setup --- ° #
env = GraphEnvironment()
# ° ------ Agent Setup ----- ° #
agent = Agent(env=env)

In [ ]:
# ° ------ TRAIN ------ ° #
if TRAIN:
    # Training
    agent.grid_search()

In [ ]:
!zip -r logs.zip /kaggle/working/logs/
FileLink(r'logs.zip')

In [ ]:
# ° ------ TEST ------ ° #
if TEST:
    # To change the detection algorithm, or the dataset, on which the model
    # will be tested, please refer to the class HyperParams in the file
    # src/utils/utils.py, changing the values of the variables:
    # - GRAPH_NAME, for the dataset
    # - DETECTION_ALG, for the detection algorithm

    # To change the model path, please refer to the class FilePaths in the
    # file src/utils/utils.py
    model_path = FilePaths.TRAINED_MODEL.value

    # Change the beta and tau parameters on which the model will be tested
    betas = [HyperParams.BETA.value] # [1,3,5]
    taus = [0.3, 0.5, 0.8]

    # get communty target
    community_target = agent.env.community_target
    for beta in betas:
        for tau in taus:
            print("* * Node Hiding with beta = {} and tau = {}".format(beta, tau))
            test(agent=agent, model_path=model_path, beta=beta, tau=tau)
            print("* * Community Hiding with beta = {} and tau = {}".format(beta, tau))
            community_deception(agent=agent, community_target=community_target, beta=beta, tau=tau, model_path=model_path)
            print("*"*50)

In [ ]:
# !zip -r test.zip /kaggle/working/test/
# FileLink(r'test.zip')